In [38]:
import pandas as pd
from transformers import AutoTokenizer
import Functions
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import re
import Functions

In [39]:
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, roc_auc_score,make_scorer
from scipy.stats import ks_2samp

In [40]:
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
import tensorflow as tf

In [42]:
df = pd.read_csv('/Users/user/Documents/Spam_Detection/Data/data_clear.csv')

In [43]:
df=df.drop(columns='index',axis=1)

In [44]:
X = df['Message']
Y = df['Category']

In [45]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5152    This is the 2nd time we have tried 2 contact u...
5153                 Will ü b going to esplanade fr home?
5154    Pity, * was in mood for that. So...any other s...
5155    The guy did some bitching but I acted like i'd...
5156                           Rofl. Its true to its name
Name: Message, Length: 5157, dtype: object


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [47]:
print(X_train)

3512                Old Orchard near univ. How about you?
3500    Thanks for your ringtone order, reference numb...
668        What is this 'hex' place you talk of? Explain!
3903                  Its ok, called mom instead have fun
2895                             Webpage s not available!
                              ...                        
4931                 But we havent got da topic yet rite?
3264    K.. I yan jiu liao... Sat we can go 4 bugis vi...
1653    (No promises on when though, haven't even gott...
2607     Can you do a mag meeting this avo at some point?
2732    Fuck cedar key and fuck her (come over anyway ...
Name: Message, Length: 4125, dtype: object


In [48]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)


train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test.values
))

In [49]:
# Définition de la taille des batchs
BATCH_SIZE = 64  # Ajustable selon la mémoire disponible

# Ajout du batching aux datasets TensorFlow
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [50]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "bert-base-uncased"  # Ou "mistralai/Mistral-7B-Instruct"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

history = model.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
